# Explications des functions + exemples

### Ordre conseillé:
- **pipe (`|`)**: Créer un lien: fd pour envoyer + fd pour recevoir
- **fork**: Créer un nouveau process en dupliquant l'actuel
- **dup2 pipe fd**: dupliquer les fd du pipe dans l'enfant
- **open**: Ouvrir les fichiers d'input+output
- **dup2 fichier fd**: dupliquer les fd des fichiers dans l'enfant
- **execve**: Lancer les executables

In [33]:
#some definitions
pipe_ex = "pipe_example.c"
fork_ex = "fork_example.c"
fds_ex = "fds_example.c"
dup2_one = "dup2_one.c"
dup2_two = "dup2_two.c"
execve_ex = "execve_example.c"

## pipe

```c
#include <unistd.h>

int pipe(int pipefd[2]);
// Remplie pipefd[2] avec les deux fd
// Renvoie -1 en cas d'erreur

#### Petite structure pratique 😎
```c
typedef struct s_pipefd
{
	int	read_fd;
	int	write_fd;
}	t_pipefd;

In [22]:
from IPython.display import Markdown, display
with open(pipe_ex) as f:
    code = f.read()
display(Markdown(f"```c\n{code}\n```"))

```c
#include <unistd.h>
#include <stdio.h>

typedef struct s_pipefd
{
	int	read_fd;
	int	write_fd;
}	t_pipefd;

int	main(void)
{
	t_pipefd	fds;
	int			result;
	char		buffer[100];

	result = pipe((int *) &fds);

	printf("resultat: %d\n", result);
	printf("fd pour read: %d\n", fds.read_fd);
	printf("fd pour write: %d\n\n", fds.write_fd);

	write(fds.write_fd, "writing in pipe", 16);
	int	n_char = read(fds.read_fd, buffer, 16);
	buffer[n_char] = '\0';
	printf("from pipe: |%s|\n", buffer);
	return (0);
}

```

In [17]:
%%bash -s "$pipe_ex"
cc "$1" && ./a.out

resultat: 0
fd pour read: 3
fd pour write: 4

from pipe: |writing in pipe|


## fork

```c
#include <unistd.h>

pid_t fork(void);
// Duplique le process actuel (duplique toute les variables, les fd, le call stack... TOUT)
// Sont indépendant: les free, les close, les variables
// renvoie 0 seulement dans l'enfant
// renvoie -1 en cas d'érreur

In [ ]:
with open(fork_ex) as f:
    code = f.read()
display(Markdown(f"```c\n{code}\n```"))

```c
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

void	action_parent()
{
	printf("Hello from parent!\n");
}
void	action_enfant()
{
	printf("Hello from child!\n");
}

int main()
{
	int	pid;

	pid = fork();
	if (pid < 0)
	{
		perror("fork failed");
		return (EXIT_FAILURE);
	}
	else if (pid == 0)
		action_enfant();
	else
		action_parent();
	return (EXIT_SUCCESS);
}

```

In [11]:
%%bash -s "$fork_ex"
cc "$1" && ./a.out

Hello from parent!
Hello from child!


## dup2

```c
#include <unistd.h>

int dup2(int oldfd, int newfd);
// Duplique le descripteur de fichier `oldfd` vers `newfd`
// Si `newfd` est déjà ouvert, il est fermé d’abord automatiquement
// Après, `oldfd` et `newfd` pointent vers **la même cible**
// Utilisé souvent pour rediriger stdin (0), stdout (1), stderr (2)

// Renvoie `newfd` en cas de succès
// Renvoie -1 en cas d'erreur

In [ ]:
with open(dup2_one) as f:
    code = f.read()
display(Markdown(f"```c\n{code}\n```"))

```c
#include "peerclass.h"

int	main(void)
{
	int	fd = open("trace.log", O_RDONLY);
	if (fd == -1)
		return (EXIT_FAILURE);

	print_open_fds();

	dup2(fd, STDIN_FILENO);
	print_open_fds();
	
	return (EXIT_SUCCESS);
}

```

In [31]:
%%bash -s "$dup2_one"
cc "$1" print_fd.c && ./a.out

Open file descriptors:
FD 0 -> pipe:[697874]
FD 1 -> pipe:[697875]
FD 2 -> pipe:[697876]
FD 3 -> /home/sarintt/Coding/42/peerclass/trace.log
FD 4 -> /proc/35518/fd

Open file descriptors:
FD 0 -> /home/sarintt/Coding/42/peerclass/trace.log
FD 1 -> pipe:[697875]
FD 2 -> pipe:[697876]
FD 3 -> /home/sarintt/Coding/42/peerclass/trace.log
FD 4 -> /proc/35518/fd



In [ ]:
with open(dup2_two) as f:
    code = f.read()
display(Markdown(f"```c\n{code}\n```"))

```c
#include "peerclass.h"

bool	replace_fd(int this_fd, int by_fd)
{
	int	ret_dup2;
	int	ret_close;

	if (by_fd < 0)
		return (false);
	ret_dup2 = dup2(by_fd, this_fd);
	ret_close = close(by_fd);
	if (ret_dup2 == -1 || ret_close == -1)
		return (false);
	return (true);
}

int	main(void)
{
	int	fd = open("trace.log", O_RDONLY);
	print_open_fds();

	printf("replacing stdin by trace.log...\n\n");
	replace_fd(STDIN_FILENO, fd);

	print_open_fds();
	return (EXIT_SUCCESS);
}

```

In [29]:
%%bash -s "$dup2_two"
cc "$1" print_fd.c && ./a.out

Open file descriptors:
FD 0 -> pipe:[697577]
FD 1 -> pipe:[697578]
FD 2 -> pipe:[697579]
FD 3 -> /home/sarintt/Coding/42/peerclass/trace.log
FD 4 -> /proc/35455/fd

replacing stdin by trace.log...

Open file descriptors:
FD 0 -> /home/sarintt/Coding/42/peerclass/trace.log
FD 1 -> pipe:[697578]
FD 2 -> pipe:[697579]
FD 3 -> /proc/35455/fd



## execve

```c
#include <unistd.h>

int execve(const char *pathname, char *const argv[], char *const envp[]);
// Remplace le process courant par un **nouveau programme** (définitif, pas de retour)
// Ne revient **jamais** si succès (le code après ne s'exécute pas)
// Utilisé souvent après fork() dans l’enfant

// `pathname` : chemin absolu vers l'exécutable (ex: "/bin/ls")
// `argv`     : arguments du programme (argv[0] = nom du programme)
// `envp`     : tableau des variables d’environnement (ex: environ)

// Renvoie -1 seulement en cas d'erreur (exécutable introuvable, permission refusée)

In [36]:
with open(execve_ex) as f:
    code = f.read()
display(Markdown(f"```c\n{code}\n```"))

```c
#include "peerclass.h"

int	main(void)
{
	char *a;
	a = malloc (42);
	a[1] = 42;
	execve("/bin/ls", (char*[]){"ls", NULL}, NULL);
	printf("Si j'arrive ici c'est que execve a fail\n");
	return (EXIT_FAILURE);
}

```

In [37]:
%%bash -s "$execve_ex"
cc "$1" && ./a.out

a.out
dup2_one.c
dup2_two.c
execve_example.c
fork_example.c
peerclass.h
peerpex.ipynb
pipe_example.c
pipex
print_fd.c
trace.log
